In [8]:
import pandas as pd
import numpy as np
import os

import wrangle as w
from env import hostname, user, password
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler,QuantileTransformer
import warnings
warnings.filterwarnings("ignore")

In [66]:
# def get_zillow_data():
#    # acquire required tables into one dataframe
#     file='zillow_df.csv'
#     if os.path.isfile(filename):
#         return pd.read_csv(filename)
#     else:
#         sql_quert = p'''SELECT
#             prop.*,
#             predictions_2017.logerror,
#             predictions_2017.transactiondate,
#             air.airconditioningdesc,
#             arch.architecturalstyledesc,
#             build.buildingclassdesc,
#             heat.heatingorsystemdesc,
#             landuse.propertylandusedesc,
#             story.storydesc,
#             construct.typeconstructiondesc
#         FROM properties_2017 prop
#         JOIN (
#             SELECT parcelid, MAX(transactiondate) max_transactiondate
#             FROM predictions_2017
#             GROUP BY parcelid
#         ) pred USING(parcelid)
#         JOIN predictions_2017 ON pred.parcelid = predictions_2017.parcelid
#                               AND pred.max_transactiondate = predictions_2017.transactiondate
#         LEFT JOIN airconditioningtype air USING (airconditioningtypeid)
#         LEFT JOIN architecturalstyletype arch USING (architecturalstyletypeid)
#         LEFT JOIN buildingclasstype build USING (buildingclasstypeid)
#         LEFT JOIN heatingorsystemtype heat USING (heatingorsystemtypeid)
#         LEFT JOIN propertylandusetype landuse USING (propertylandusetypeid)
#         LEFT JOIN storytype story USING (storytypeid)
#         LEFT JOIN typeconstructiontype construct USING (typeconstructiontypeid)
#         WHERE prop.latitude IS NOT NULL
#           AND prop.longitude IS NOT NULL
#           AND predictions_2017.transactiondate like "2017%%" ''')
#         df = pd.read_sql(sql_query, get_connection('zillow'))
#         df.to_csv(filename, index=False)
#         return df
import os
import pandas as pd

def get_zillow_data():
    # Define the filename
    filename = 'zillow_df.csv'
    
    # Check if the file exists
    if os.path.isfile(filename):
        return pd.read_csv(filename)
    else:
        # SQL query to retrieve the required data
        sql_query = '''
        SELECT
            prop.*,
            predictions_2017.logerror,
            predictions_2017.transactiondate,
            air.airconditioningdesc,
            arch.architecturalstyledesc,
            build.buildingclassdesc,
            heat.heatingorsystemdesc,
            landuse.propertylandusedesc,
            story.storydesc,
            construct.typeconstructiondesc
        FROM properties_2017 prop
        JOIN (
            SELECT parcelid, MAX(transactiondate) max_transactiondate
            FROM predictions_2017
            GROUP BY parcelid
        ) pred USING(parcelid)
        JOIN predictions_2017 ON pred.parcelid = predictions_2017.parcelid
                              AND pred.max_transactiondate = predictions_2017.transactiondate
        LEFT JOIN airconditioningtype air USING (airconditioningtypeid)
        LEFT JOIN architecturalstyletype arch USING (architecturalstyletypeid)
        LEFT JOIN buildingclasstype build USING (buildingclasstypeid)
        LEFT JOIN heatingorsystemtype heat USING (heatingorsystemtypeid)
        LEFT JOIN propertylandusetype landuse USING (propertylandusetypeid)
        LEFT JOIN storytype story USING (storytypeid)
        LEFT JOIN typeconstructiontype construct USING (typeconstructiontypeid)
        WHERE prop.latitude IS NOT NULL
          AND prop.longitude IS NOT NULL
          AND predictions_2017.transactiondate LIKE '2017%%'
        '''
        # Retrieve data from the database and save it as a CSV file
        df = pd.read_sql(sql_query, get_connection('zillow'))
        df.to_csv(filename, index=False)
        return df


In [15]:
df = w.get_zillow_data()
df.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,1727539,14297519,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,...,6.059063e+13,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,1387261,17052889,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,...,6.111001e+13,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,11677,14186244,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,6.059022e+13,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,2288172,12177905,NaN,NaN,NaN,3.0,4.0,NaN,8.0,3.0,...,6.037300e+13,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,1970746,10887214,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,6.037124e+13,0.006940,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN


In [17]:
df.describe()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,logerror
count,7.738000e+04,7.738000e+04,24953.000000,206.000000,50.000000,77380.000000,77380.000000,15.000000,49671.000000,76771.000000,...,17560.000000,172.0,7.726800e+04,7.737900e+04,77380.0,7.737800e+04,77375.000000,2886.000000,7.713600e+04,77380.000000
mean,1.495126e+06,1.300715e+07,1.813289,7.388350,679.720000,2.299134,3.053489,3.933333,6.534638,2.316871,...,1.434738,1.0,1.893160e+05,4.901375e+05,2016.0,3.010969e+05,5995.566592,14.087318,6.049674e+13,0.016628
std,8.609057e+05,3.481368e+06,2.967894,2.734542,689.703546,0.996657,1.139103,0.258199,1.721933,0.979761,...,0.544569,0.0,2.300886e+05,6.534477e+05,0.0,4.925990e+05,7622.887460,2.185663,1.535252e+12,0.170190
min,3.490000e+02,1.071186e+07,1.000000,2.000000,38.000000,0.000000,0.000000,3.000000,1.000000,1.000000,...,1.000000,1.0,4.400000e+01,1.000000e+03,2016.0,1.610000e+02,19.920000,3.000000,6.037101e+13,-4.655420
25%,7.520500e+05,1.153830e+07,1.000000,7.000000,273.000000,2.000000,2.000000,4.000000,6.000000,2.000000,...,1.000000,1.0,8.426500e+04,2.070000e+05,2016.0,8.550400e+04,2715.585000,14.000000,6.037311e+13,-0.024377
50%,1.497870e+06,1.253155e+07,1.000000,7.000000,515.000000,2.000000,3.000000,4.000000,6.000000,2.000000,...,1.000000,1.0,1.364995e+05,3.589760e+05,2016.0,2.033835e+05,4450.720000,15.000000,6.037603e+13,0.006627
75%,2.240480e+06,1.421184e+07,1.000000,7.000000,796.500000,3.000000,4.000000,4.000000,8.000000,3.000000,...,2.000000,1.0,2.187875e+05,5.690030e+05,2016.0,3.668022e+05,6927.800000,15.000000,6.059042e+13,0.039204
max,2.982274e+06,1.676893e+08,13.000000,21.000000,3560.000000,18.000000,16.000000,4.000000,12.000000,18.000000,...,6.000000,1.0,1.142179e+07,4.906124e+07,2016.0,4.895220e+07,586639.300000,99.000000,4.830301e+14,5.262999


In [25]:
df.isnull().sum()

id                              0
parcelid                        0
airconditioningtypeid       52427
architecturalstyletypeid    77174
basementsqft                77330
                            ...  
buildingclassdesc           77365
heatingorsystemdesc         27941
propertylandusedesc             0
storydesc                   77330
typeconstructiondesc        77158
Length: 68, dtype: int64

In [37]:
df.shape

(77380, 68)

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77380 entries, 0 to 77379
Data columns (total 68 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            77380 non-null  int64  
 1   parcelid                      77380 non-null  int64  
 2   airconditioningtypeid         24953 non-null  float64
 3   architecturalstyletypeid      206 non-null    float64
 4   basementsqft                  50 non-null     float64
 5   bathroomcnt                   77380 non-null  float64
 6   bedroomcnt                    77380 non-null  float64
 7   buildingclasstypeid           15 non-null     float64
 8   buildingqualitytypeid         49671 non-null  float64
 9   calculatedbathnbr             76771 non-null  float64
 10  decktypeid                    614 non-null    float64
 11  finishedfloor1squarefeet      6023 non-null   float64
 12  calculatedfinishedsquarefeet  77184 non-null  float64
 13  f

In [40]:
# Filter properties with a transaction in 2017
df_2017 = df[df['transactiondate'].str.startswith('2017')]
df.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,1727539,14297519,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,...,6.059063e+13,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,1387261,17052889,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,...,6.111001e+13,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,11677,14186244,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,6.059022e+13,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,2288172,12177905,NaN,NaN,NaN,3.0,4.0,NaN,8.0,3.0,...,6.037300e+13,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,1970746,10887214,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,6.037124e+13,0.006940,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN


Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.


In [44]:
#John G code
def missing_nulls(df):
    num_missing = df.isnull().sum()
    rows = df.shape[0]
    pct_missing = num_missing / rows
    cols_missing = pd.DataFrame({'number_missing_rows': num_missing, 'percent_rows_missing': pct_missing})
    return cols_missing

In [43]:
missing_nulls(df)

,number_missing_rows,percent_rows_missing
id,0,0.000000
parcelid,0,0.000000
airconditioningtypeid,52427,0.677526
architecturalstyletypeid,77174,0.997338
basementsqft,77330,0.999354
...,...,...
buildingclassdesc,77365,0.999806
heatingorsystemdesc,27941,0.361088
propertylandusedesc,0,0.000000
storydesc,77330,0.999354


In [46]:

df.nulls_by_column.sort_values(by="percent_rows_missing", ascending=False).head(40)

AttributeError: 'DataFrame' object has no attribute 'nulls_by_column'

2. Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

In [47]:
# calue counts by unit 

df.unitcnt.value_counts(dropna = False)

1.0      47292
NaN      26818
2.0       2018
4.0        723
3.0        525
45.0         1
42.0         1
6.0          1
237.0        1
Name: unitcnt, dtype: int64

In [49]:
# Remove unnecessary propertie, leaveonly properties that meet SU criteria

single_use = [261, 262, 263, 264, 266, 268, 273, 276, 279]
df = df[df.propertylandusetypeid.isin(single_use)]
    
    
# Restrict df to only those properties with at least 1 bath & bed and >350 sqft area
df = df[(df.bedroomcnt > 0) & (df.bathroomcnt > 0) & ((df.unitcnt<=1)|df.unitcnt.isnull()) & (df.calculatedfinishedsqft)

SyntaxError: incomplete input (724719716.py, line 8)

Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

In [57]:
def clean_missing_values(df, prop_required_column = .5, prop_required_row = .70):
    threshold = int(round(prop_required_column*len(df.index),0))
    df.dropna(axis=1, thresh=threshold, inplace=True)
    threshold = int(round(prop_required_row*len(df.columns),0))
    df.dropna(axis=0, thresh=threshold, inplace=True)
    return df

In [58]:
clean_missing_values(df)

,id,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror,transactiondate,heatingorsystemdesc,propertylandusedesc
0,1727539,14297519,3.5,4.0,NaN,3.5,3100.0,3100.0,6059.0,3.0,...,485713.0,1023282.0,2016.0,537569.0,11013.72,6.059063e+13,0.025595,2017-01-01,NaN,Single Family Residential
1,1387261,17052889,1.0,2.0,NaN,1.0,1465.0,1465.0,6111.0,1.0,...,88000.0,464000.0,2016.0,376000.0,5672.48,6.111001e+13,0.055619,2017-01-01,NaN,Single Family Residential
2,11677,14186244,2.0,3.0,NaN,2.0,1243.0,1243.0,6059.0,2.0,...,85289.0,564778.0,2016.0,479489.0,6488.30,6.059022e+13,0.005383,2017-01-01,NaN,Single Family Residential
3,2288172,12177905,3.0,4.0,8.0,3.0,2376.0,2376.0,6037.0,3.0,...,108918.0,145143.0,2016.0,36225.0,1777.51,6.037300e+13,-0.103410,2017-01-01,Central,Single Family Residential
4,1970746,10887214,3.0,3.0,8.0,3.0,1312.0,1312.0,6037.0,3.0,...,73681.0,119407.0,2016.0,45726.0,1533.89,6.037124e+13,0.006940,2017-01-01,Central,Condominium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77375,2864704,10833991,3.0,3.0,8.0,3.0,1741.0,1741.0,6037.0,3.0,...,265000.0,379000.0,2016.0,114000.0,4685.34,6.037132e+13,-0.002245,2017-09-20,Central,Condominium
77376,673515,11000655,2.0,2.0,6.0,2.0,1286.0,1286.0,6037.0,2.0,...,70917.0,354621.0,2016.0,283704.0,4478.43,6.037101e+13,0.020615,2017-09-20,Central,Single Family Residential
77377,2968375,17239384,2.0,4.0,NaN,2.0,1612.0,1612.0,6111.0,2.0,...,50683.0,67205.0,2016.0,16522.0,1107.48,6.111008e+13,0.013209,2017-09-21,NaN,Single Family Residential
77378,1843709,12773139,1.0,3.0,4.0,1.0,1032.0,1032.0,6037.0,1.0,...,32797.0,49546.0,2016.0,16749.0,876.43,6.037434e+13,0.037129,2017-09-21,Central,Single Family Residential


In [60]:
df.shape

(77304, 34)

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77304 entries, 0 to 77379
Data columns (total 34 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            77304 non-null  int64  
 1   parcelid                      77304 non-null  int64  
 2   bathroomcnt                   77304 non-null  float64
 3   bedroomcnt                    77304 non-null  float64
 4   buildingqualitytypeid         49671 non-null  float64
 5   calculatedbathnbr             76771 non-null  float64
 6   calculatedfinishedsquarefeet  77184 non-null  float64
 7   finishedsquarefeet12          73748 non-null  float64
 8   fips                          77304 non-null  float64
 9   fullbathcnt                   76771 non-null  float64
 10  heatingorsystemtypeid         49439 non-null  float64
 11  latitude                      77304 non-null  float64
 12  longitude                     77304 non-null  float64
 13  l

In [ ]:
cols_to_remove = ['id',
       'calculatedbathnbr', 'finishedsquarefeet12', 'fullbathcnt', 'heatingorsystemtypeid'
       ,'propertycountylandusecode', 'propertylandusetypeid','propertyzoningdesc', 
        'censustractandblock', 'propertylandusedesc', 'unitcnt']

In [64]:
#drop null values 
df.dropna()

,id,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror,transactiondate,heatingorsystemdesc,propertylandusedesc
3,2288172,12177905,3.0,4.0,8.0,3.0,2376.0,2376.0,6037.0,3.0,...,108918.0,145143.0,2016.0,36225.0,1777.51,6.037300e+13,-0.103410,2017-01-01,Central,Single Family Residential
4,1970746,10887214,3.0,3.0,8.0,3.0,1312.0,1312.0,6037.0,3.0,...,73681.0,119407.0,2016.0,45726.0,1533.89,6.037124e+13,0.006940,2017-01-01,Central,Condominium
6,781532,12095076,3.0,4.0,9.0,3.0,2962.0,2962.0,6037.0,3.0,...,276684.0,773303.0,2016.0,496619.0,9516.26,6.037461e+13,-0.001011,2017-01-01,Central,Single Family Residential
8,1246926,12790562,3.0,4.0,9.0,3.0,3039.0,3039.0,6037.0,3.0,...,177527.0,220583.0,2016.0,43056.0,3104.19,6.037500e+13,-0.040966,2017-01-02,Central,Single Family Residential
9,1585097,11542646,3.0,2.0,8.0,3.0,1290.0,1290.0,6037.0,3.0,...,151303.0,371361.0,2016.0,220058.0,4557.32,6.037275e+13,-0.036763,2017-01-02,Central,Condominium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77374,2274245,12412492,2.0,4.0,6.0,2.0,1633.0,1633.0,6037.0,2.0,...,125466.0,346534.0,2016.0,221068.0,4175.08,6.037555e+13,0.001082,2017-09-19,Central,Single Family Residential
77375,2864704,10833991,3.0,3.0,8.0,3.0,1741.0,1741.0,6037.0,3.0,...,265000.0,379000.0,2016.0,114000.0,4685.34,6.037132e+13,-0.002245,2017-09-20,Central,Condominium
77376,673515,11000655,2.0,2.0,6.0,2.0,1286.0,1286.0,6037.0,2.0,...,70917.0,354621.0,2016.0,283704.0,4478.43,6.037101e+13,0.020615,2017-09-20,Central,Single Family Residential
77378,1843709,12773139,1.0,3.0,4.0,1.0,1032.0,1032.0,6037.0,1.0,...,32797.0,49546.0,2016.0,16749.0,876.43,6.037434e+13,0.037129,2017-09-21,Central,Single Family Residential


In [65]:
df.isnull().sum()

id                                  0
parcelid                            0
bathroomcnt                         0
bedroomcnt                          0
buildingqualitytypeid           27633
calculatedbathnbr                 533
calculatedfinishedsquarefeet      120
finishedsquarefeet12             3556
fips                                0
fullbathcnt                       533
heatingorsystemtypeid           27865
latitude                            0
longitude                           0
lotsizesquarefeet                8171
propertycountylandusecode           0
propertylandusetypeid               0
propertyzoningdesc              26987
rawcensustractandblock              0
regionidcity                     1460
regionidcounty                      0
regionidzip                        45
roomcnt                             0
unitcnt                         26742
yearbuilt                         187
structuretaxvaluedollarcnt        102
taxvaluedollarcnt                   1
assessmentye

In [31]:
# Sort the DataFrame by transactiondate in descending order
df_2017.sort_values('transactiondate', ascending=False, inplace=True)
df_2017.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
77379,1187175,12826780,NaN,NaN,NaN,2.0,3.0,NaN,6.0,2.0,...,6.037503e+13,0.007204,2017-09-25,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
77377,2968375,17239384,NaN,NaN,NaN,2.0,4.0,NaN,NaN,2.0,...,6.111008e+13,0.013209,2017-09-21,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
77378,1843709,12773139,1.0,NaN,NaN,1.0,3.0,NaN,4.0,1.0,...,6.037434e+13,0.037129,2017-09-21,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN
77376,673515,11000655,NaN,NaN,NaN,2.0,2.0,NaN,6.0,2.0,...,6.037101e+13,0.020615,2017-09-20,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
77375,2864704,10833991,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,6.037132e+13,-0.002245,2017-09-20,Central,NaN,NaN,Central,Condominium,NaN,NaN


In [33]:
# Keep only the last transaction for each property ID
df_2017.drop_duplicates(subset='propertyid', inplace=True)
df_2017.datatype(

KeyError: Index(['propertyid'], dtype='object')

In [27]:
# Filter properties with latitude and longitude values
df_filtered = df_2017[df_2017['latitude'].notnull() & df_2017['longitude'].notnull()]

# Select required columns (propertyid, logerror, transactiondate)
df_final = df_filtered[['propertyid', 'logerror', 'transactiondate']]

# Reset the index of the final DataFrame
df_final.reset_index(drop=True, inplace=True)

KeyError: Index(['propertyid'], dtype='object')

In [35]:
df.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,1727539,14297519,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,...,6.059063e+13,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,1387261,17052889,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,...,6.111001e+13,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,11677,14186244,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,6.059022e+13,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,2288172,12177905,NaN,NaN,NaN,3.0,4.0,NaN,8.0,3.0,...,6.037300e+13,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,1970746,10887214,1.0,NaN,NaN,3.0,3.0,NaN,8.0,3.0,...,6.037124e+13,0.006940,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN
